In [7]:
using BenchmarkTools: @ballocated
using LinearAlgebra: I, norm, triu, tril, tr, diagm, diag, qr, eigvals, istriu, dot
using CairoMakie

In [24]:
function arnoldi(A, q1, kmax)
    H = zeros(kmax, kmax)
    Q = zeros(size(q1, 1), kmax)

    Q[:, 1] = q1 / norm(q1)
    for k = 1:kmax
        if k > 1
            Q[:, k] = q1/ H[k, k-1]
        end

        q1 = A * Q[:, k]
        for i = 1:k
            H[i,k] = dot(Q[:,i], q1)
            q1 -= H[i, k] * Q[:, i]
        end

        if k < kmax
            H[k+1, k] = norm(q1)
        end
    end
    return H
end

m = 5
A = rand(m,m)
A = A * A' 
display(A)
H = arnoldi(A, randn(m), m)
display(H)

display(istriu(H[2 : end, 1 : (end - 1)]))
display(eigvals(H) ≈ eigvals(A))

5×5 Matrix{Float64}:
 0.713131  1.34871  0.913088  1.00638  0.996093
 1.34871   2.9192   1.52344   1.85025  2.14215
 0.913088  1.52344  1.40556   1.46447  1.11526
 1.00638   1.85025  1.46447   2.32263  1.34365
 0.996093  2.14215  1.11526   1.34365  1.58368

5×5 Matrix{Float64}:
 0.236755  0.920109  6.66134e-16  -2.8727e-15   4.5079e-14
 0.920109  6.86643   2.0088       -1.90438e-14  2.88827e-13
 0.0       2.0088    1.42898       0.23303      6.4811e-14
 0.0       0.0       0.23303       0.398241     0.0220556
 0.0       0.0       0.0           0.0220556    0.013794

true

true

In [20]:
function lanczos(A, q1, kmax)
    T = zeros(kmax, kmax)
    r = copy(q1[:, 1])
    beta = norm(r)
    q1 = zeros(size(A, 1))
    
    for k = 1:kmax
        q0 = copy(q1)
        q1 = r/beta
        r = A * q1
        alpha = dot(q1, r)
        T[k, k] = alpha
        if k > 1
            T[k-1, k] = beta
            T[k, k-1] = beta
        end
        r = r - alpha*q1 - beta * q0
        beta = norm(r)
    end
    return diag(T), diag(T,1)
end

m = 5
A = rand(m,m)
A = A * A' 
a, b = lanczos(A, randn(m), m)

H = diagm(-1 => b, 0 => a, 1 => b)

eigvals(H) ≈ eigvals(A) 

true

In [44]:
m = 10
h_m = 10
A = rand(m,m)
A = A * A' 

H_A = arnoldi(A, randn(m), h_m)

a, b = lanczos(A, randn(m), h_m)
H_L = diagm(-1 => b, 0 => a, 1 => b)

display(sort(eigvals(A), rev=true))
display(sort(eigvals(H_A), rev=true))
display(sort(eigvals(H_L), rev=true))

# eigs(H_A)


10-element Vector{Float64}:
 25.572980599599113
  2.1008756395892125
  1.3427066679802486
  1.1456804447877986
  1.0134442596792603
  0.42088531298615933
  0.37488553449994855
  0.16119617753307805
  0.08581254825565977
  0.0013721003841208304

10-element Vector{Float64}:
 25.572980599599106
  2.1008756395892125
  1.34270666798025
  1.1456804447878
  1.0134442596792632
  0.42088531298615867
  0.37488553449994944
  0.16119617753307747
  0.08581254825566093
  0.0013721003841211197

10-element Vector{Float64}:
 25.572980599599106
  2.1008756395892156
  1.3427066685233195
  1.1456804557261269
  1.0134442630018525
  0.42089144522447913
  0.375780511716897
  0.1611963079008721
  0.08581263466379627
  0.0013721336912092346